In [1]:
# For Reference 
# Date;Time;BTC/USD Bid;BTC/USD Bid Size;BTC/USD Ask;BTC/USD Ask Size;BTC/EUR Bid;BTC/EUR Bid Size;BTC/EUR Ask;BTC/EUR Ask Size;ETH/BTC Bid;ETH/BTC Bid Size;ETH/BTC Ask;ETH/BTC Ask Size;EUR/USD Bid;EUR/USD Bid Size;EUR/USD Ask;EUR/USD Ask Size;ETH/USD Bid;ETH/USD Bid Size;ETH/USD Ask;ETH/USD Ask Size;ETH/EUR Bid;ETH/EUR Bid Size;ETH/EUR Ask;ETH/EUR Ask Size

In [1]:
# Note datetime is the module name, but datetime is a class two. To call the datetime class use datetime.datetime
import datetime

In [3]:
def remove_from_str(s: str, substrings_to_remove: set[str]) -> str:
    """
    Pretty inefficient but whatever.
    Use regex for speedup.
    """
    ret = s
    for sub in substrings_to_remove:
        ret = ret.replace(sub, "")
    return(ret)

In [2]:
def next_interval(t: datetime.datetime) -> datetime.datetime:
    """
    Return the next 5 mintute interval
    """
    return(t + datetime.timedelta(minutes=5))

# print(datetime.datetime(2021, 9, 13, 12, 10))
# print(next_interval(datetime.datetime(2021, 9, 13, 12, 10)))

2021-09-13 12:10:00
2021-09-13 12:15:00


In [4]:
chars_to_remove = {'$', '€', ',', "BTC"}
f = open("Assignment 1 - Data.txt", 'r')
text = f.read()

rows = text.split('\n')

In [5]:
d = dict()
indexes = dict()

columns = rows[0].split(';')

for trade_type in columns[2:]:
    d[trade_type] = dict()

for i in range(len(columns)):
    indexes[i] = columns[i] 


In [6]:
for row in rows[1:]:
    split = row.split(';')
    # print(split)
    for i in range(len(split)):
        if i == 0:
            date = split[i]
        elif i == 1:
            time_of_day = split[i]
        else:
            trade_time = datetime.datetime.strptime(date + ' ' + time_of_day, "%m/%d/%Y %H:%M %p")
            trade_type = indexes[i]
            d[trade_type][trade_time] = float(remove_from_str(split[i], chars_to_remove))
print(d)
            

{'BTC/USD Bid': {datetime.datetime(2021, 9, 13, 12, 0): 36633.24, datetime.datetime(2021, 9, 13, 12, 5): 36508.57, datetime.datetime(2021, 9, 13, 12, 10): 36052.58, datetime.datetime(2021, 9, 13, 12, 15): 37121.22, datetime.datetime(2021, 9, 13, 12, 20): 38573.84, datetime.datetime(2021, 9, 13, 12, 25): 37529.96, datetime.datetime(2021, 9, 13, 12, 30): 36714.84, datetime.datetime(2021, 9, 13, 12, 35): 37369.43, datetime.datetime(2021, 9, 13, 12, 40): 36947.05, datetime.datetime(2021, 9, 13, 12, 45): 38201.21, datetime.datetime(2021, 9, 13, 12, 50): 39728.69, datetime.datetime(2021, 9, 13, 12, 55): 39215.87, datetime.datetime(2021, 9, 13, 1, 0): 40201.33, datetime.datetime(2021, 9, 13, 1, 5): 40874.98, datetime.datetime(2021, 9, 13, 1, 10): 40497.13, datetime.datetime(2021, 9, 13, 1, 15): 40030.89, datetime.datetime(2021, 9, 13, 1, 20): 39874.97, datetime.datetime(2021, 9, 13, 1, 25): 39917.0, datetime.datetime(2021, 9, 13, 1, 30): 40219.93, datetime.datetime(2021, 9, 13, 1, 35): 40646.